# ICS mitre mapping

This notebook attempts to visualize the mapping between several ICS threat actors and conducted attacks to the detections as to discover the attack.

This notebook uses data from the MITRE ATT&CK ICS framework using the MITRE STIX data in the [attack-stix-data](https://github.com/mitre-attack/attack-stix-data/tree/master) repository.

## Requirements

There are some libraries that we need and with which the code was tested

* **taxii2-client**: To talk to the taxii server
* **stix2**: To handle the stix2 encapsulated data
* **networkx**: To create and work with graphs
* **bokeh**: To visualize graphs
* **pygraphviz**: Additional library to structure the graph

**NOTE:** Pygraphviz is actually a base package that needs to be installed via the terminal, for Ubuntu and other debian based systems run ```sudo apt-get install graphviz graphviz-dev```.

For other systems consult [pygraphviz install guide](https://github.com/pygraphviz/pygraphviz/blob/main/INSTALL.txt) 

In [193]:
%pip install taxii2-client==2.3.0
%pip install stix2==3.0.1
%pip install networkx==2.6.3
%pip install bokeh==2.3.3
%pip install pandas==1.3.5
%pip install pygraphviz==1.12
%pip install selenium
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Initial setup and retrieving data
The debugging level is lowered to only show critical messages, and we use the following url to work with: 
```https://cti-taxii.mitre.org/taxii/```


In [194]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)
server = Server("https://cti-taxii.mitre.org/taxii/")

# Retrieve API root
api_root = server.api_roots[0]

# Select ICS ATT&CK data collection
MITRE_COLLECTION_ID = api_root.collections[3].id

Now get the actual collection, based on the selected collection ID

In [195]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
MITRE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MITRE_COLLECTION_ID + "/")
MC_DATA = TAXIICollectionSource(MITRE_COLLECTION)

Also get the mitigation (course of action), detections, attacker groups, software and technique data. We retreive it as a mapping of unique IDs and human readable names

In [196]:
MC_MITIGATIONS = MC_DATA.query(Filter("type", "=", "course-of-action"))
mc_mitigation_mapping = {}
for x in MC_MITIGATIONS:
  mc_mitigation_mapping[x['id']] = x['name']

In [197]:
MC_TECHNIQUES = MC_DATA.query(Filter("type", "=", "attack-pattern"))
mc_technique_mapping = {}
for x in MC_TECHNIQUES:
  mc_technique_mapping[x['id']] = x['name']

In [198]:
MC_DETECTIONS = MC_DATA.query(Filter("type", "=", "x-mitre-data-component"))
mc_detection_mapping = {}
for x in MC_DETECTIONS:
  mc_detection_mapping[x['id']] = x['name']

In [199]:
MC_INTRUSION_SET = MC_DATA.query(Filter("type", "=", "intrusion-set"))
mc_intrusion_set_mapping = {}
for x in MC_INTRUSION_SET:
  mc_intrusion_set_mapping[x['id']] = x['name']

In [200]:
MC_MALWARE = MC_DATA.query(Filter("type", "=", "malware"))
mc_malware_mapping = {}
for x in MC_MALWARE:
  mc_malware_mapping[x['id']] = x['name']

Let's now get the relationships between threat actors, software, techniques, and detections.
The data is then put into a pandas dataframe with the human readable names as well for better context

In [204]:
import pprint
import pandas as pd

MC_RELATIONS = MC_DATA.query(Filter("type","=","relationship"))

mc_ot_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx,x in enumerate(MC_RELATIONS):
  if 'relationship_type' in x:
    if x['relationship_type'] == 'uses' and 'intrusion-set' in x['source_ref'] and 'malware' in x['target_ref']:
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_intrusion_set_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_malware_mapping[x['target_ref']])
    elif x['relationship_type'] == 'uses' and 'malware' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_malware_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_technique_mapping[x['target_ref']])
    elif x['relationship_type'] == 'detects':
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_detection_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_technique_mapping[x['target_ref']])
      

df = pd.DataFrame(data=mc_ot_data)
temp_df = df[df['source_human'] == "Industroyer"]
display(temp_df)
print(temp_df.size)
# temp_df[temp_df['Destination'].str.endswith("2700ec")]


,Source,Destination,source_human,target_human
21,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--2fedbe69-581f-447d-8a78-32ee7d...,Industroyer,Remote System Information Discovery
298,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--2bb4d762-bf4a-4bc3-9318-15cc6a...,Industroyer,Loss of Protection
316,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--24a9253e-8948-4c98-b751-8e2aee...,Industroyer,Command-Line Interface
317,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--d5a69cfb-fc2a-46cb-99eb-74b236...,Industroyer,Remote System Discovery
318,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--1b22b676-9347-4c55-9a35-ef0dc6...,Industroyer,Denial of Service
319,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--25dfc8ad-bd73-4dfd-84a9-3c3d38...,Industroyer,Device Restart/Shutdown
320,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--1c478716-71d9-46a4-9a53-fa5d57...,Industroyer,Block Serial COM
321,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--138979ba-0430-4de6-a128-2fc0b0...,Industroyer,Loss of View
322,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--40b300ba-f553-48bf-862e-9471b2...,Industroyer,Unauthorized Command Message
323,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--a81696ef-c106-482c-8f80-59c30f...,Industroyer,Loss of Control


96


In [205]:
mc_ot_data_mitigations = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}

for idx,x in enumerate(MC_RELATIONS):
    if 'relationship_type' in x:
        if x['relationship_type'] == 'mitigates' and 'course-of-action' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
            mc_ot_data_mitigations['Source'].append(x['source_ref'])
            mc_ot_data_mitigations['Destination'].append(x['target_ref'])
            mc_ot_data_mitigations['source_human'].append(mc_mitigation_mapping[x['source_ref']])
            mc_ot_data_mitigations['target_human'].append(mc_technique_mapping[x['target_ref']])
        elif x['relationship_type'] == 'uses' and 'malware' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
            mc_ot_data_mitigations['Source'].append(x['source_ref'])
            mc_ot_data_mitigations['Destination'].append(x['target_ref'])
            mc_ot_data_mitigations['source_human'].append(mc_malware_mapping[x['source_ref']])
            mc_ot_data_mitigations['target_human'].append(mc_technique_mapping[x['target_ref']])

mitigations_df = pd.DataFrame(data=mc_ot_data_mitigations)
mitigations_df.head()

,Source,Destination,source_human,target_human
0,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--fc5fda7e-6b2c-4457-b036-759896...,Human User Authentication,Modify Program
1,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--09a61657-46e1-439e-b3ed-3e4556...,Human User Authentication,Modify Controller Tasking
2,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--097924ce-a9a9-4039-8591-e0deed...,Human User Authentication,Modify Parameter
3,course-of-action--e0d38502-decb-481d-ad8b-b8f0...,attack-pattern--fc5fda7e-6b2c-4457-b036-759896...,Authorization Enforcement,Modify Program
4,course-of-action--e0d38502-decb-481d-ad8b-b8f0...,attack-pattern--09a61657-46e1-439e-b3ed-3e4556...,Authorization Enforcement,Modify Controller Tasking


## Visualizing all data

In [206]:
# https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html
from bokeh.io import output_notebook, show, save
# required to display visuals inline
output_notebook()

Loading BokehJS ...

In [207]:
from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save, export_png
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx

# Create the base graph from the pandas dataframe
DG = nx.from_pandas_edgelist(df, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
all_mappings.update(mc_intrusion_set_mapping)
all_mappings.update(mc_malware_mapping)

nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}

for node in DG.nodes:
    if node.startswith('intrusion-set'):
      node_types[node] = 0
    elif node.startswith('malware'):
      node_types[node] = 1 
    elif node.startswith('attack-pattern'):
      node_types[node] = 2
    else:
      node_types[node] = 3

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#000000','#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
# export_png(plot, filename=f"{title}.png")
# save(plot, filename=f"{title}.png")

In [208]:
class MiterTechniqueHelper:
    def __init__(self, stix_data_store, mitre_technique_id) -> None:
        self.stix_store = stix_data_store
        self._technique_id = mitre_technique_id.upper()
        self._stix_id = self._stixid_from_id()
        self._technique_name = self._name_from_id()
        self._technique_tactics = self._tactics_from_id()
        self._datasources = self._datasources_from_id()
        self._mitigations = self._mitigations_from_id()
        self._targets = self._targets_from_id()

In [209]:
from pyvis.network import Network


# Directed_Graph= nx.from_pandas_edgelist(df, source='Source', target='Destination', create_using=nx.DiGraph)
Directed_graph = nx.DiGraph()
"""
Tactics for constructing the graph
Initially only put the intrusion sets in the graph
Then add the software and link them to the intrusion sets
After this Link the techniques to the software.
And lastly link the edges from the techniques to the detections / data sources


Use stix idx as id in the graph and assign label as human readable name
"""
height = 2500 
width = (height / 9)*16
width_cell = width / 5

# Define positions for nodes
positions = {
    'IS': {'x': 0, 'y': 0},  # Intrusion Sets
    'SW': {'x': 0, 'y': 100},  # Malware
    'AP': {'x': 100, 'y': 0},  # Attack Patterns
    'DT': {'x': 100, 'y': 100}  # Detection Techniques
}
x_positions = {'IS':0*width_cell,'SW':1*width_cell,'AP':2*width_cell,'DT':3*width_cell, 'MT':4*width_cell}
y_positions = {'IS':0,'SW':0,'AP':0,'DT':0, 'MT':0}


IS_nodes = df[df['Source'].str.startswith('intrusion-set')]['Source'].unique()
SW_nodes = df[df['Source'].str.startswith('malware')]['Source'].unique()
AP_nodes = df[df['Destination'].str.startswith('attack-pattern')]['Destination'].unique()
DT_nodes = df[df['Source'].str.startswith('x-mitre-data-component')]['Source'].unique()
print(len(IS_nodes),len(SW_nodes),len(AP_nodes),len(DT_nodes))

IS_len = height / len(IS_nodes)
SW_len = height / len(SW_nodes)
AP_len = height /   (1/2* len(AP_nodes))
DT_len = height / len(DT_nodes)

for node in IS_nodes:
    node_title = df[df['Source'] == node]['source_human'].iloc[0]
    Directed_graph.add_node(
        node, 
        label=node_title, 
        color='red', 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        level = 0,
        x=x_positions['IS'], 
        y=y_positions['IS'],
        shape='triangle'
    )
    y_positions['IS'] += IS_len

for node in SW_nodes:
    node_title = df[df['Source'] == node]['source_human'].iloc[0]
    Directed_graph.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='black', 
        level = 1,
        x=x_positions['SW'], 
        y=y_positions['SW'],
    )
    y_positions['SW'] += SW_len

for node in AP_nodes:
    node_title = df[df['Destination'] == node]['target_human'].iloc[0]
    Directed_graph.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='green', 
        level = 2,
        x=x_positions['AP'], 
        y=y_positions['AP'],
        shape='star'
    )
    y_positions['AP'] += AP_len 
    
for node in DT_nodes:
    node_title = df[df['Source'] == node]['target_human'].iloc[0]
    Directed_graph.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='blue', 
        level = 3,
        x=x_positions['DT'], 
        y=y_positions['DT'],
        shape='triangle'
    )
    y_positions['DT'] += DT_len
    
subset = df[['Source', 'Destination']]
edge_list = [tuple(x) for x in subset.to_numpy()]
Directed_graph.add_edges_from(edge_list, color='skyblue')

# graph_node_degrees = dict(Directed_graph.degree())
# graph_scale = 2
# for node_id, node_degree in graph_node_degrees.items():
#     if node_degree < 5:
#         graph_scale = 3
#     Directed_graph.nodes[node_id]['weight'] = node_degree * graph_scale
#     Directed_graph.nodes[node_id]['size'] = node_degree * graph_scale

pyvis_nt = Network(notebook=True, directed = True, filter_menu=True, height=f"%(height)spx"%{'height':height}, width=f"%(width)spx"%{'width':width})

pyvis_nt.from_nx(Directed_graph)
pyvis_nt.toggle_physics(False)
pyvis_nt.toggle_hide_edges_on_drag(True)
# pyvis_nt.show_buttons()
pyvis_nt.save_graph('DG.html')

9 21 80 34


In [210]:
Mitigations_DAG = nx.DiGraph()
width_cell = width / 3
# x_positions = {'SW':0*width_cell,'AP':1*width_cell,'MT':2*width_cell}
y_positions = {'SW':0,'AP':0,'MT':0}


SW_nodes = mitigations_df[mitigations_df['Source'].str.startswith('malware')]['Source'].unique()
AP_nodes = mitigations_df[mitigations_df['Destination'].str.startswith('attack-pattern')]['Destination'].unique()
MT_nodes = mitigations_df[mitigations_df['Source'].str.startswith('course-of-action')]['Source'].unique()
print(len(SW_nodes),len(AP_nodes),len(MT_nodes))

SW_len = height / len(SW_nodes)
AP_len = height / len(AP_nodes)
MT_len = height / len(MT_nodes)


for node in SW_nodes:
    node_title = mitigations_df[mitigations_df['Source'] == node]['source_human'].iloc[0]
    Mitigations_DAG.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='black', 
        level = 1,
        x=x_positions['SW'], 
        y=y_positions['SW'],
    )
    y_positions['SW'] += SW_len

for node in AP_nodes:
    node_title = mitigations_df[mitigations_df['Destination'] == node]['target_human'].iloc[0]
    Mitigations_DAG.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='green', 
        level = 2,
        x=x_positions['AP'], 
        y=y_positions['AP'],
        shape='star'
    )
    y_positions['AP'] += AP_len 
    
for node in MT_nodes:
    node_title = mitigations_df[mitigations_df['Source'] == node]['source_human'].iloc[0]
    Mitigations_DAG.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='blue', 
        level = 3,
        x=x_positions['MT'], 
        y=y_positions['MT'],
        shape='triangle'
    )
    y_positions['MT'] += MT_len
    
subset = mitigations_df[['Source', 'Destination']]
edge_list = [tuple(x) for x in subset.to_numpy()]
Mitigations_DAG.add_edges_from(edge_list, color='skyblue')

# graph_node_degrees = dict(Directed_graph.degree())
# graph_scale = 2
# for node_id, node_degree in graph_node_degrees.items():
#     if node_degree < 5:
#         graph_scale = 3
#     Directed_graph.nodes[node_id]['weight'] = node_degree * graph_scale
#     Directed_graph.nodes[node_id]['size'] = node_degree * graph_scale

pyvis_nt = Network(notebook=True, directed = True, filter_menu=True, height=f"%(height)spx"%{'height':height}, width=f"%(width)spx"%{'width':width})

pyvis_nt.from_nx(Mitigations_DAG)
pyvis_nt.toggle_physics(False)
pyvis_nt.toggle_hide_edges_on_drag(True)
# pyvis_nt.show_buttons()
pyvis_nt.save_graph('DG_mitigations.html')

21 81 51


## Data analysis


In [211]:
# https://networkx.org/documentation/stable/auto_examples/drawing/plot_degree.html
detection_data = {'Detection':[],'Degree':[]}
for idx, deg in DG.degree:
    if idx.startswith('x-mitre-data'):
        human_detection = mc_detection_mapping[idx]
        detection_data['Detection'].append(human_detection)
        detection_data['Degree'].append(deg) 
detections = pd.DataFrame(data=detection_data)
detections = detections.sort_values(by=['Degree'], ascending=False, ignore_index=True)
# detections.to_string(index=False)

In [212]:
pd.set_option('display.max_rows', None)
ttp_data = {'Technique':[],'Degree':[]}
ttps = {}
for idx, deg in DG.degree:
    if idx.startswith('attack-pattern'):
        human_detection = mc_technique_mapping[idx]
        ttp_data['Technique'].append(human_detection)
        ttp_data['Degree'].append(deg) 
ttps = pd.DataFrame(data=ttp_data)
ttps.sort_values(by=['Degree'], ascending=False, ignore_index=True)

,Technique,Degree
0,Masquerading,12
1,Lateral Tool Transfer,12
2,Service Stop,12
3,Indicator Removal on Host,10
4,User Execution,10
5,Remote Services,10
6,Remote System Information Discovery,9
7,Remote System Discovery,9
8,Unauthorized Command Message,8
9,Data from Local System,8


In [213]:
pd.set_option('display.max_rows', None)
mitigation_data = {'Mitigation':[],'Degree':[]}
for idx, deg in Mitigations_DAG.degree:
    if idx.startswith('course-of-action'):
        human_detection = mc_mitigation_mapping[idx]
        mitigation_data['Mitigation'].append(human_detection)
        mitigation_data['Degree'].append(deg) 
mitigations = pd.DataFrame(data=mitigation_data)
mitigations = mitigations.sort_values(by=['Degree'], ascending=False, ignore_index=True)

In [251]:
coverage_list = list(detections['Detection'].iloc[:5])
display(list(detections['Detection'].iloc[:5]))

# df[df['target_human'].isin(coverage_list)]
df_coverage = df.query('source_human.isin(@coverage_list)')
print(df_coverage['Destination'].unique().size)


['Network Traffic Content',
 'Application Log Content',
 'Network Traffic Flow',
 'Process Creation',
 'Command Execution']

60


In [252]:
mitigation_coverage = list(mitigations['Mitigation'].iloc[:10])
display(list(mitigations['Mitigation'].iloc[:10]))

# df[df['target_human'].isin(coverage_list)]
df_coverage = mitigations_df.query('source_human.isin(@mitigation_coverage)')
print(df_coverage['Destination'].unique().size)

['Network Segmentation',
 'Network Allowlists',
 'Software Process and Device Authentication',
 'Human User Authentication',
 'Communication Authenticity',
 'Access Management',
 'Filter Network Traffic',
 'Audit',
 'Authorization Enforcement',
 'Out-of-Band Communications Channel']

52


In [216]:

MT_nodes = mitigations_df[mitigations_df['Source'].str.startswith('course-of-action')]['Source'].unique()
print(len(MT_nodes))
MT_len = height / len(MT_nodes)

for node in MT_nodes:
    node_title = mitigations_df[mitigations_df['Source'] == node]['source_human'].iloc[0]
    Directed_graph.add_node(
        node, 
        label=node_title, 
        title=f"%(title)s, Stix: %(stix)s"%{'title':node_title, "stix":node}, 
        color='white', 
        level = 4,
        x=x_positions['MT'], 
        y=y_positions['MT'],
        shape='ellipse',
    )
    y_positions['MT'] += MT_len

mitigation_edges = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}

for idx,x in enumerate(MC_RELATIONS):
    if 'relationship_type' in x:
        if x['relationship_type'] == 'mitigates' and 'course-of-action' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
            mitigation_edges['Source'].append(x['source_ref'])
            mitigation_edges['Destination'].append(x['target_ref'])
            mitigation_edges['source_human'].append(mc_mitigation_mapping[x['source_ref']])
            mitigation_edges['target_human'].append(mc_technique_mapping[x['target_ref']])
mitigation_edges = pd.DataFrame(data=mitigation_edges)

subset = mitigation_edges[['Source', 'Destination']]
edge_list = [tuple(x) for x in subset.to_numpy()]
Directed_graph.add_edges_from(edge_list, color='skyblue')

pyvis_nt = Network(notebook=True, directed = True, filter_menu=True, height=f"%(height)spx"%{'height':height}, width=f"%(width)spx"%{'width':width})

pyvis_nt.from_nx(Directed_graph)
pyvis_nt.toggle_physics(False)
pyvis_nt.toggle_hide_edges_on_drag(True)

pyvis_nt.save_graph('DG_full.html')

51


In [244]:
coverage_data = {'Action':[],'Degree':[]}
for idx, deg in DG.degree:
    if idx.startswith('x-mitre-data'):
        human_detection = mc_detection_mapping[idx]
        coverage_data['Action'].append(human_detection)
        coverage_data['Degree'].append(deg) 
for idx, deg in Mitigations_DAG.degree:
    if idx.startswith('course-of-action'):
        human_mitigation = mc_mitigation_mapping[idx]
        coverage_data['Action'].append(human_mitigation)
        coverage_data['Degree'].append(deg) 
coverage_df= pd.DataFrame(data=coverage_data)
coverage_df = coverage_df.sort_values(by=['Degree'], ascending=False, ignore_index=True)

In [246]:
mc_ot_full = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}

for idx,x in enumerate(MC_RELATIONS):
    if 'relationship_type' in x:
        if x['relationship_type'] == 'mitigates' and 'course-of-action' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
           mc_ot_full['Source'].append(x['source_ref'])
           mc_ot_full['Destination'].append(x['target_ref'])
           mc_ot_full['source_human'].append(mc_mitigation_mapping[x['source_ref']])
           mc_ot_full['target_human'].append(mc_technique_mapping[x['target_ref']])
        elif x['relationship_type'] == 'uses' and 'malware' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
           mc_ot_full['Source'].append(x['source_ref'])
           mc_ot_full['Destination'].append(x['target_ref'])
           mc_ot_full['source_human'].append(mc_malware_mapping[x['source_ref']])
           mc_ot_full['target_human'].append(mc_technique_mapping[x['target_ref']])
        elif x['relationship_type'] == 'uses' and 'intrusion-set' in x['source_ref'] and 'malware' in x['target_ref']:
           mc_ot_full['Source'].append(x['source_ref'])
           mc_ot_full['Destination'].append(x['target_ref'])
           mc_ot_full['source_human'].append(mc_intrusion_set_mapping[x['source_ref']])
           mc_ot_full['target_human'].append(mc_malware_mapping[x['target_ref']])
        elif x['relationship_type'] == 'detects':
           mc_ot_full['Source'].append(x['source_ref'])
           mc_ot_full['Destination'].append(x['target_ref'])
           mc_ot_full['source_human'].append(mc_detection_mapping[x['source_ref']])
           mc_ot_full['target_human'].append(mc_technique_mapping[x['target_ref']])
full_df = pd.DataFrame(data=mc_ot_full)
full_df.head()

,Source,Destination,source_human,target_human
0,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--fc5fda7e-6b2c-4457-b036-759896...,Human User Authentication,Modify Program
1,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--09a61657-46e1-439e-b3ed-3e4556...,Human User Authentication,Modify Controller Tasking
2,course-of-action--66cfe23e-34b6-4583-b178-ed6a...,attack-pattern--097924ce-a9a9-4039-8591-e0deed...,Human User Authentication,Modify Parameter
3,course-of-action--e0d38502-decb-481d-ad8b-b8f0...,attack-pattern--fc5fda7e-6b2c-4457-b036-759896...,Authorization Enforcement,Modify Program
4,course-of-action--e0d38502-decb-481d-ad8b-b8f0...,attack-pattern--09a61657-46e1-439e-b3ed-3e4556...,Authorization Enforcement,Modify Controller Tasking


In [253]:

coverage_list = list(coverage_df['Action'].iloc[:20])
# display(list(coverager'action'].iloc[:5]))
display(list(coverage_df['Action'].iloc[:20]))

# df[df['target_human'].isin(coverage_list)]
df_coverage = full_df.query('source_human.isin(@coverage_list)')
print(df_coverage['Destination'].unique().size)

['Network Traffic Content',
 'Application Log Content',
 'Network Segmentation',
 'Network Traffic Flow',
 'Network Allowlists',
 'Process Creation',
 'Software Process and Device Authentication',
 'Human User Authentication',
 'Communication Authenticity',
 'Command Execution',
 'Filter Network Traffic',
 'Access Management',
 'Audit',
 'Device Alarm',
 'Authorization Enforcement',
 'Out-of-Band Communications Channel',
 'Code Signing',
 'Data Backup',
 'Process History/Live Data',
 'Update Software']

75


## Further refinement of visualization (not functional yet)

To do further visualization, we'll convert the NetworkX graph to a PyGraphViz object and then some arranging of the different node/vertex types into columns for a better connection overview.

In [ ]:
import pygraphviz as pgv

A = nx.nx_agraph.to_agraph(DG)

show()

## Attack techniques for the Industroyer and Triton / Trisis Malware

The code below is to retrieve the techniques used by the pieces of software (mitre) / malware (STIX), with the collection of Data Sources / detections which allow for the detection of the corresponding technique.

In [3]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)
server = Server("https://cti-taxii.mitre.org/taxii/")

api_root = server.api_roots[0]

MITRE_COLLECTION_ID_ICS = api_root.collections[3].id
MITRE_COLLECTION_ID_ENT = api_root.collections[0].id

In [4]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
MITRE_COLLECTION_ICS = Collection(ATTACK_STIX_COLLECTIONS + MITRE_COLLECTION_ID_ICS + "/")
MITRE_COLLECTION_ENT = Collection(ATTACK_STIX_COLLECTIONS + MITRE_COLLECTION_ID_ENT + "/")
MC_DATA_ICS = TAXIICollectionSource(MITRE_COLLECTION_ICS)
MC_DATA_ENT = TAXIICollectionSource(MITRE_COLLECTION_ENT)

In [6]:
def relations(relation_type, data_source):
    MC_RELATIONS = data_source.query(Filter("type","=",relation_type))
    mc_relations ={}
    for x in MC_RELATIONS:
        mc_relations[x['id']] = x['name']
    return mc_relations

In [7]:
MC_MALWARE_ICS = relations("malware", MC_DATA_ICS)
MC_INTRUSION_SETS_ICS = relations("intrusion-set", MC_DATA_ICS)
MC_TECHNIQUES_ICS = relations("attack-pattern", MC_DATA_ICS)
MC_DETECTIONS_ICS = relations("x-mitre-data-component", MC_DATA_ICS)
MC_MITIGATIONS_ICS = relations("course-of-action", MC_DATA_ICS)

MC_MALWARE_ENT = relations("malware", MC_DATA_ENT)
MC_INTRUSION_SETS_ENT = relations("intrusion-set", MC_DATA_ENT)
MC_TECHNIQUES_ENT = relations("attack-pattern", MC_DATA_ENT)
MC_DETECTIONS_ENT = relations("x-mitre-data-component", MC_DATA_ENT)
MC_MITIGATIONS_ENT = relations("course-of-action", MC_DATA_ENT)

In [10]:
import pandas as pd

def generate_relations(base_data, malware_relations, intrusion_relations, technique_relations, detection_relations, mitigation_relations):
    MC_RELATIONS = base_data.query(Filter("type","=","relationship"))

    mc_structured_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
    for idx, x in enumerate(MC_RELATIONS):
        if 'relationship_type' in x:
            if x['relationship_type'] == 'uses' and 'intrusion-set' in x['source_ref'] and 'malware' in x['target_ref']:
                mc_structured_data['Source'].append(x['source_ref'])
                mc_structured_data['Destination'].append(x['target_ref'])
                mc_structured_data['source_human'].append(intrusion_relations[x['source_ref']])
                mc_structured_data['target_human'].append(malware_relations[x['target_ref']])
            elif x['relationship_type'] == 'uses' and 'malware' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
                mc_structured_data['Source'].append(x['source_ref'])
                mc_structured_data['Destination'].append(x['target_ref'])
                mc_structured_data['source_human'].append(malware_relations[x['source_ref']])
                mc_structured_data['target_human'].append(technique_relations[x['target_ref']])
            elif x['relationship_type'] == 'detects':
                mc_structured_data['Source'].append(x['source_ref'])
                mc_structured_data['Destination'].append(x['target_ref'])
                mc_structured_data['source_human'].append(detection_relations[x['source_ref']])
                mc_structured_data['target_human'].append(technique_relations[x['target_ref']])
            elif x['relationship_type'] == 'mitigates' and 'course-of-action' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
                mc_structured_data['Source'].append(x['source_ref'])
                mc_structured_data['Destination'].append(x['target_ref'])
                mc_structured_data['source_human'].append(mitigation_relations[x['source_ref']])
                mc_structured_data['target_human'].append(technique_relations[x['target_ref']])
            elif x['relationship_type'] == 'mitigates' and 'course-of-action' in x['source_ref'] and 'malware' in x['target_ref']:
                mc_structured_data['Source'].append(x['source_ref'])
                mc_structured_data['Destination'].append(x['target_ref'])
                mc_structured_data['source_human'].append(mitigation_relations[x['source_ref']])
                mc_structured_data['target_human'].append(malware_relations[x['target_ref']])
    return pd.DataFrame(data=mc_structured_data)

In [33]:
df_ent = generate_relations(MC_DATA_ENT, MC_MALWARE_ENT, MC_INTRUSION_SETS_ENT, MC_TECHNIQUES_ENT, MC_DETECTIONS_ENT, MC_MITIGATIONS_ENT)
df_ics = generate_relations(MC_DATA_ICS, MC_MALWARE_ICS, MC_INTRUSION_SETS_ICS, MC_TECHNIQUES_ICS, MC_DETECTIONS_ICS, MC_MITIGATIONS_ICS)
df_ent

,Source,Destination,source_human,target_human
0,malware--4e6464d2-69df-4e56-8d4c-1973f84d7b80,attack-pattern--43e7dc91-05b2-474c-b9ac-2ed4fe...,Mispadu,Process Injection
1,malware--554e010d-726b-439d-9a1a-f60fff0cc109,attack-pattern--04fd5427-79c7-44ea-ae13-11b247...,GLASSTOKEN,Standard Encoding
2,malware--29a0bb87-1162-4c83-9834-2a98a876051b,attack-pattern--e6919abc-99f9-4c6c-95a5-14761e...,BUSHWALK,Ingress Tool Transfer
3,malware--bcaae558-9697-47a2-9ec7-c75000ddf58c,attack-pattern--3ccef7ae-cb5e-48f6-8302-897105...,FRAMESTING,Deobfuscate/Decode Files or Information
4,malware--5911d2ca-64f6-49b3-b94f-29b5d185085c,attack-pattern--707399d6-ab3e-4963-9315-d9d381...,SocGholish,System Network Configuration Discovery
...,...,...,...,...
12034,intrusion-set--8f5e8dc7-739d-4f5e-a8a1-a66e004...,malware--fde50aaa-f5de-4cb8-989a-babb57d6a704,Cleaver,Net Crawler
12035,intrusion-set--8f5e8dc7-739d-4f5e-a8a1-a66e004...,malware--c0c45d38-fe57-4cd4-b2b2-9ecd0ddd4ca9,Cleaver,TinyZBot
12036,intrusion-set--a0cb9370-e39b-44d5-9f50-ef78e41...,malware--95047f03-4811-4300-922e-1ba937d53a61,Axiom,Hikit
12037,intrusion-set--2e5d3a83-fe00-41a5-9b60-237efc8...,malware--b42378e0-f147-496f-992a-26a49705395b,Moafee,PoisonIvy


Get all rows which are related to Industroyer and triton/trisis malware

In [28]:
ent_techniques


,Source,Destination,source_human,target_human
4809,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--4fe28b27-b13c-453e-a386-c2ef36...,Industroyer,Protocol Tunneling
5411,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--e358d692-23c0-4a31-9eb6-ecc13a...,Industroyer,Remote System Discovery
6052,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--df8b2a25-8bdf-4856-953c-a04372...,Industroyer,Web Protocols
6053,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--a782ebe2-daba-42c7-bc82-e8e9d9...,Industroyer,Multi-hop Proxy
6054,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--92d7da27-2d91-488e-a00c-059dc1...,Industroyer,Exfiltration Over C2 Channel
6055,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--960c3c86-1480-4d72-b4e0-8c242e...,Industroyer,Compromise Host Software Binary
6056,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--b17a1a56-e99c-403c-8948-561df0...,Industroyer,Valid Accounts
6057,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--707399d6-ab3e-4963-9315-d9d381...,Industroyer,System Network Configuration Discovery
6058,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--2bee5ffb-7a7a-4119-b1f2-158151...,Industroyer,Application or System Exploitation
6059,malware--e401d4fe-f0c9-44f0-98e6-f93487678808,attack-pattern--b3d682b6-98f2-4fb0-aa3b-b4df00...,Industroyer,Obfuscated Files or Information


In [105]:
def get_detections(technique_data, full_data, filter):
    return_dict = {}
    for idx, x in technique_data.iterrows():
        detections = full_data[(full_data['Destination'] == x['Destination']) & (full_data["Source"].str.contains(filter))]
        return_dict[x['target_human']] = list(detections['source_human'])
    return return_dict

In [112]:
ent_techniques = df_ent[df_ent['Source'] == "malware--e401d4fe-f0c9-44f0-98e6-f93487678808"]
ics_techniques = df_ics[df_ics['Source'] == "malware--e401d4fe-f0c9-44f0-98e6-f93487678808"]

ent_detections = get_detections(ent_techniques, df_ent, 'x-mitre-data-component')
ics_detections = get_detections(ics_techniques, df_ics, 'x-mitre-data-component')

ent_mitigations = get_detections(ent_techniques, df_ent, 'course-of-action')
ics_mitigations = get_detections(ics_techniques, df_ics, 'course-of-action')

final_dict = {
    "ent_detections": ent_detections,
    "ics_detections": ics_detections,
    "ent_mitigations": ent_mitigations,
    "ics_mitigations": ics_mitigations
}

import json
with open('output_industroyer.json', 'w+') as file:
    json.dump(final_dict, file)

In [111]:
ics_techniques = df_ics[df_ics['source_human'] == "Triton"]

ics_detections = get_detections(ics_techniques, df_ics, 'x-mitre-data-component')

ics_mitigations = get_detections(ics_techniques, df_ics, 'course-of-action')

final_dict = {
    "ics_detections": ics_detections,
    "ics_mitigations": ics_mitigations
}

import json
with open('output_triton.json', 'w+') as file:
    json.dump(final_dict, file)